**OS path**
- 학원 노트북용 --> C:/Users/Playdata/Playdata_anl/1st_Project/dataset/
- 개인 노트북용 --> C:/Users/Brian/Desktop/playdata/Playdata_anl/1st_Project/dataset/

In [1]:
import pandas as pd
import numpy as np

**만들어놓은 최종 데이터 셋 불러오기**

In [2]:
final_data = pd.read_csv("C:/Users/Brian/Desktop/playdata/Playdata_anl/1st_Project/dataset/subway_weather_pop_final.csv", dtype = {'transfer':str, 'gate':str})
final_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
final_data.head()

,date,line,station_num,station_name,boarding,05~06,06~07,07~08,08~09,09~10,...,total_19,total_20,total_21,total_22,total_23,total_5,total_6,total_7,total_8,total_9
0,2017-01-01,1호선,150,서울역,승차,470,286,397,786,1421,...,13965.0479,14464.2979,15162.6506,16336.8151,16005.3908,16668.3599,16699.1654,15776.5716,14370.7991,13410.9693
1,2017-01-01,1호선,150,서울역,하차,278,880,859,964,1407,...,13965.0479,14464.2979,15162.6506,16336.8151,16005.3908,16668.3599,16699.1654,15776.5716,14370.7991,13410.9693
2,2017-01-02,1호선,150,서울역,하차,356,2366,4852,9813,4784,...,25181.3513,26227.9772,27251.8970,27774.0245,32401.8704,32491.9088,31453.9491,29285.2527,25878.3191,24701.9018
3,2017-01-02,1호선,150,서울역,승차,411,536,2297,3665,2103,...,25181.3513,26227.9772,27251.8970,27774.0245,32401.8704,32491.9088,31453.9491,29285.2527,25878.3191,24701.9018
4,2017-01-03,1호선,150,서울역,하차,303,1719,4452,10406,4863,...,52916.4980,52575.2426,52218.9502,52409.0778,47157.2329,47120.9495,47685.4421,47704.0643,45729.1596,44860.2133


- target 변수로 사용할 컬럼들의 이름을 바꿔주도록 하겠다.
    - 나중에 모델링 할 때, 반복적인 작업을 해주려면 컬럼명을 비슷한 유형으로 통일시켜주는 것이 좋기 때문에!!

In [3]:
final_data.rename(columns = {'05~06':'subway_board_5', '06~07':'subway_board_6', '07~08':'subway_board_7', '08~09':'subway_board_8',
                             '09~10':'subway_board_9', '10~11':'subway_board_10', '11~12':'subway_board_11', '12~13':'subway_board_12',
                             '13~14':'subway_board_13', '14~15':'subway_board_14', '15~16':'subway_board_15', '16~17':'subway_board_16',
                             '17~18':'subway_board_17', '18~19':'subway_board_18', '19~20':'subway_board_19', '20~21':'subway_board_20', 
                             '21~22':'subway_board_21', '22~23':'subway_board_22', '23~24':'subway_board_23', '00~01':'subway_board_0'},
                             inplace = True)

# 컬럼명이 제대로 변경되었는지 확인
final_data.iloc[:2, 5:25]

,subway_board_5,subway_board_6,subway_board_7,subway_board_8,subway_board_9,subway_board_10,subway_board_11,subway_board_12,subway_board_13,subway_board_14,subway_board_15,subway_board_16,subway_board_17,subway_board_18,subway_board_19,subway_board_20,subway_board_21,subway_board_22,subway_board_23,subway_board_0
0,470,286,397,786,1421,1459,2623,2851,3124,2304,3178,3610,3858,3376,3384,2537,3914,2660,917,112
1,278,880,859,964,1407,1622,1681,2070,2345,2606,2682,2743,3074,2492,2120,1798,1618,962,551,179


- str 타입으로 되어있는 date 변수를 datetime 타입으로 변환시켜주기

In [4]:
# 'str' to 'datetime'
final_data['date'] = pd.to_datetime(final_data['date'], format = '%Y-%m-%d')
final_data['date']

0        2017-01-01
1        2017-01-01
2        2017-01-02
3        2017-01-02
4        2017-01-03
            ...    
560689   2019-12-26
560690   2019-12-27
560691   2019-12-27
560692   2019-12-28
560693   2019-12-28
Name: date, Length: 560694, dtype: datetime64[ns]

- 최종 데이터 셋에 해당하는 날짜만 따로 만들어서, 바로 옆에 weekdays(요일) 컬럼을 만들어주겠다.
    - 이렇게 따로 만들어 준 뒤, inner join을 하는 것이 속도적인 측면에서 훨씬 더 빠르다고 판단했기 때문!!

In [5]:
dates = pd.date_range('2017-01-01', '2019-12-31')
df_dates = pd.DataFrame(dates)
df_dates.rename(columns = {0:'date'}, inplace = True)
df_dates['date'] = pd.to_datetime(df_dates['date'], format = '%Y-%m-%d')
df_dates

,date
0,2017-01-01
1,2017-01-02
2,2017-01-03
3,2017-01-04
4,2017-01-05
...,...
1090,2019-12-27
1091,2019-12-28
1092,2019-12-29
1093,2019-12-30


In [6]:
len(set(final_data['date']))

1095

In [7]:
import datetime

result = []
for i in range(len(df_dates)):
    result.append(df_dates['date'][i].weekday())

len(result)

1095

In [8]:
df_dates['weekdays'] = result
df_dates

,date,weekdays
0,2017-01-01,6
1,2017-01-02,0
2,2017-01-03,1
3,2017-01-04,2
4,2017-01-05,3
...,...,...
1090,2019-12-27,4
1091,2019-12-28,5
1092,2019-12-29,6
1093,2019-12-30,0


- 위 결과를 보면, '날짜'에 해당하는 '요일'이 잘 들어간 것을 확인할 수 있다.

**final_data + df_dates (데이터 결합)**
- date 변수를 조인키로 사용하여 inner join을 수행해 줄 것이다.
    - 그러면 final_data에 weekdays(요일) 변수가 붙을 것이다!!

In [9]:
final_data2 = pd.merge(final_data, df_dates, how = 'inner', on = 'date')
final_data2

,date,line,station_num,station_name,boarding,subway_board_5,subway_board_6,subway_board_7,subway_board_8,subway_board_9,...,total_20,total_21,total_22,total_23,total_5,total_6,total_7,total_8,total_9,weekdays
0,2017-01-01,1호선,150,서울역,승차,470,286,397,786,1421,...,14464.2979,15162.6506,16336.8151,16005.3908,16668.3599,16699.1654,15776.5716,14370.7991,13410.9693,6
1,2017-01-01,1호선,150,서울역,하차,278,880,859,964,1407,...,14464.2979,15162.6506,16336.8151,16005.3908,16668.3599,16699.1654,15776.5716,14370.7991,13410.9693,6
2,2017-01-01,6호선,2629,삼각지,하차,36,72,53,99,134,...,20936.3477,20834.5510,21175.1083,20945.6270,20677.1184,20328.2989,19835.2743,20298.7933,21139.1331,6
3,2017-01-01,6호선,2629,삼각지,승차,58,79,45,75,102,...,20936.3477,20834.5510,21175.1083,20945.6270,20677.1184,20328.2989,19835.2743,20298.7933,21139.1331,6
4,2017-01-01,6호선,2628,효창공원앞,하차,24,92,79,94,169,...,40696.6729,35855.0342,33073.6851,25660.7875,28418.8617,30522.5438,32648.4301,36860.1874,41269.1932,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560689,2019-12-31,3호선,341,경찰병원,하차,13,108,253,1201,580,...,13562.4516,13939.9637,14566.4176,16078.1886,16340.5373,16113.6454,16411.4788,15533.9862,15048.5180,1
560690,2019-12-31,3호선,342,오금,하차,0,61,167,580,187,...,25061.4871,26536.7724,27780.5161,28351.8037,28091.0819,28228.3091,28605.8111,28499.9459,28681.9841,1
560691,2019-12-31,3호선,342,오금,승차,120,243,922,1182,534,...,25061.4871,26536.7724,27780.5161,28351.8037,28091.0819,28228.3091,28605.8111,28499.9459,28681.9841,1
560692,2019-12-31,3호선,321,충무로,승차,0,0,0,0,0,...,16934.8080,17990.8998,18136.3871,19083.2652,19898.6936,19387.2799,17740.7458,16464.9906,15457.8757,1


In [10]:
len(final_data)

560694

In [11]:
final_data2[['date', 'weekdays']]

,date,weekdays
0,2017-01-01,6
1,2017-01-01,6
2,2017-01-01,6
3,2017-01-01,6
4,2017-01-01,6
...,...,...
560689,2019-12-31,1
560690,2019-12-31,1
560691,2019-12-31,1
560692,2019-12-31,1


**<참고>**
- ```weekday()```: 요일 반환
    - 0: 월, 1 :화, 2 :수, 3 :목, 4 :금, 5 :토, 6 :일

In [12]:
final_data2['weekdays'].value_counts()

0    80394
6    80392
1    80390
5    79880
4    79880
3    79880
2    79878
Name: weekdays, dtype: int64

**인구 데이터 부분에 해당하는 변수들을 반올림**
- ```np.around()```를 사용해서 정수 부분만 남기도록 하겠다.

In [13]:
from tqdm.notebook import tqdm

for i in tqdm(list(range(5, 24)) + [0]):
    final_data2[f'below_20_{i}'] = np.around(final_data2[f'below_20_{i}'])
    final_data2[f'pop_20_{i}'] = np.around(final_data2[f'pop_20_{i}'])
    final_data2[f'pop_30_{i}'] = np.around(final_data2[f'pop_30_{i}'])
    final_data2[f'pop_40_{i}'] = np.around(final_data2[f'pop_40_{i}'])
    final_data2[f'pop_50_{i}'] = np.around(final_data2[f'pop_50_{i}'])
    final_data2[f'pop_60_{i}'] = np.around(final_data2[f'pop_60_{i}'])
    final_data2[f'over_70_{i}'] = np.around(final_data2[f'over_70_{i}'])
    final_data2[f'total_{i}'] = np.around(final_data2[f'total_{i}'])
    
final_data2.iloc[0:5, 207:367]

,below_20_0,below_20_10,below_20_11,below_20_12,below_20_13,below_20_14,below_20_15,below_20_16,below_20_17,below_20_18,...,total_19,total_20,total_21,total_22,total_23,total_5,total_6,total_7,total_8,total_9
522752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**요일을 의미하는 'weekdays' 변수가 추가된 최종 데이터 셋을 csv 파일로 내보내기**

In [14]:
final_data2.to_csv("C:/Users/Brian/Desktop/playdata/Playdata_anl/1st_Project/dataset/final_data.csv")